## Импорт библиотек

In [1]:
import os
import cv2
from copy import deepcopy
from pathlib import Path
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import numpy as np
from statistics import mode

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

print(tf.__version__)

2.4.1


## Загрузка датафрейма с данными

In [ ]:
data_folder = Path('./Data')  # root directory

df = pd.read_csv(data_folder / 'df_cleared.csv')  # train dataset

In [ ]:
# Словарь эмоций
emotion_mapping = dict(enumerate(df['emotion'].unique()))
emotion_mapping

## Preprocessing function

In [ ]:
def preprocess_vgg(image_):
    
    """image_ - tensor of size (1, H, W, 3)
    return: image, with preprocess_input applied (..., version=2) from keras_vggface"""
    
    preprocessed_image = utils.preprocess_input(image_, version=2)

    return preprocessed_image

## Загрузка моделей

In [ ]:
class emotion_model:
    def __init__(self, config:str):
        self.config = config
        self.model = tf.keras.models.load_model(self.config)
        
    def predict_model(self, image, preprocessing_function, preprocess=True):
        image = np.expand_dims(image, axis=0)
        if preprocess:
            image = preprocessing_function(image)
        predicts = self.model.predict(image)
        emotion = emotion_mapping[np.argmax(predicts)]        
        return emotion

In [ ]:
model_1 = emotion_model('./saved_model_resnet50_trial/1')

In [ ]:
model_2 = emotion_model('./saved_model_xception_trial/1')

In [ ]:
model_3 = emotion_model('./saved_model_bit_trial/1')

In [ ]:
model_4 = emotion_model('./saved_model_efficientnetb1_trial/1')

## Подключение камеры

In [ ]:
cam = cv2.VideoCapture(0)

cam.set(cv2.CAP_PROP_FPS, 24)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 640) # ширина кадра -- 640 пикселей
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480) # высота кадра -- 480 пикселей

## Детектор лиц

In [ ]:
modelFile = "./res10_300x300_ssd_iter_140000.caffemodel"
configFile = "./deploy.prototxt"
detector = cv2.dnn.readNetFromCaffe(configFile, modelFile)

## Запуск модели

In [ ]:
while(True):
    ret, bgr_frame = cam.read()
    rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)
    faces = cv2.dnn.blobFromImage(image=rgb_frame, scalefactor=1.0, 
                                  size=(300, 300), 
                                  mean=(104.0, 177.0, 123.0),
                                  swapRB=True)
    
    bgr_frame_copy = deepcopy(bgr_frame)
    frameHeight = rgb_frame.shape[0]
    frameWidth = rgb_frame.shape[1]
    conf_threshold = 0.8  # порог точности детектора
    detector.setInput(faces)
    detections = detector.forward()
    
    bboxes = []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)

            face_bbox_bgr = bgr_frame[y1:y2, x1:x2]
            face_bbox_rgb = cv2.cvtColor(face_bbox_bgr, cv2.COLOR_BGR2RGB)
    
            img = image.img_to_array(cv2.resize(face_bbox_rgb, dsize=(224, 224), interpolation=cv2.INTER_CUBIC))
            
            emotion_1 = model_1.predict_model(img, preprocess_vgg)
            emotion_2 = model_2.predict_model(img, tf.keras.applications.xception.preprocess_input)
            emotion_3 = model_3.predict_model(img, preprocess=False, preprocessing_function=None)
            emotion_4 = model_4.predict_model(img, preprocess=False, preprocessing_function=None)
            emotion = mode([emotion_1, emotion_2, emotion_3, emotion_3])

            cv2.rectangle(bgr_frame_copy, (x1, y1), (x2, y2), (0,255,0), 1)
            cv2.putText(bgr_frame_copy, emotion, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow("facial emotion recognition", bgr_frame_copy)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break